In [4]:
# Librerias
#import pandas as pd
import requests
from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.common.by import By
#import numpy as np
#from selenium import webdriver  
#from selenium.common.exceptions import NoSuchElementException

#from selenium.webdriver.common.keys import Keys
#import time

In [5]:
# Headers para poder acceder a la pagina
header = {
    'User-Agent': 'Chrome 108.0.5359.125',
    'Accept-Language': 'es'
}

# Recetas Gratis

### Obtención de los links de las diferentes categorías

In [80]:
url_recetas_gratis = "https://www.recetasgratis.net"
pagina_recetas_gratis = requests.get(url_recetas_gratis, headers=header)
soup_recetas_gratis = BeautifulSoup(pagina_recetas_gratis.content, 'html.parser')

# Obtener los tags <a> de las diferentes categorias
titulos_categorias_recetas_gratis = []
contenedores_categorias_recetas_gratis = soup_recetas_gratis.find_all('div', class_="categoria ga")
for categoria in range(len(contenedores_categorias_recetas_gratis)):
    titulos_categorias_recetas_gratis.append(contenedores_categorias_recetas_gratis[categoria].find('a', class_="titulo"))

# Obtener el link de cada categoría   
links_categorias = []
for categoria in range(len(titulos_categorias_recetas_gratis)):
    lista_links = list(titulos_categorias_recetas_gratis)[categoria]
    links_categorias.append(lista_links.get('href'))
    
links_categorias

['https://www.recetasgratis.net/Recetas-de-Aperitivos-tapas-listado_receta-1_1.html',
 'https://www.recetasgratis.net/Recetas-de-Arroces-cereales-listado_receta-9_1.html',
 'https://www.recetasgratis.net/Recetas-de-Aves-caza-listado_receta-11_1.html',
 'https://www.recetasgratis.net/Recetas-de-Carne-listado_receta-10_1.html',
 'https://www.recetasgratis.net/Recetas-de-Cocteles-bebida-listado_receta-15_1.html',
 'https://www.recetasgratis.net/Recetas-de-Consejos-de-cocina-listado_receta-6146_1.html',
 'https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html',
 'https://www.recetasgratis.net/Recetas-de-Guisos-Potage-listado_receta-19_1.html',
 'https://www.recetasgratis.net/Recetas-de-Huevos-lacteos-listado_receta-18_1.html',
 'https://www.recetasgratis.net/Recetas-de-Legumbres-listado_receta-8_1.html',
 'https://www.recetasgratis.net/Recetas-de-Mariscos-listado_receta-13_1.html',
 'https://www.recetasgratis.net/Recetas-de-Pan-bolleria-listado_receta-16_1.html',
 'http

### Obtención de las diferentes hojas para todas las categorías y las diferentes receeas pertenecientes a x categoría

In [84]:
final = False
num_categoria = 0
links = []

# Por cada categoria se recogen las diferentes paginas que existen para esa categoria
for i in links_categorias:
    url_prueba = i
    
    # Comrpobar si ha llegado al final de la lista de links de las categorias
    if num_categoria == len(links_categorias)-1:
        final = True
    else:
        final = False
    
    links.append(url_prueba)
    # Bucle que recorre todas las paginas en una categoria, obteniendo los diferentes links de cada hoja 
    while(final==False):
        
        pags_mirar2 = requests.get(url_prueba)
        soup = BeautifulSoup(pags_mirar2.content, 'html.parser')
        
        # Obtener el marcador de siguiente hoja
        icon_next = soup.find('a', {"class":"next ga"})
        # Si el icono resulta nulo significa que nos encontramos al final de las hojas para esa categoria
        if icon_next is None:
            final = True
            links.append(url_prueba)
        else:
            # Si existe, se obtiene el link de la siguiente hoja
            icon_next = soup.find('a', {"class":"next ga"}).get("href")
            links.append(icon_next)
            url_prueba = icon_next
            
    num_categoria+=1
    
# Navega entre las diferentes hojas de cada categoria y obtiene los links de las diferentes
nombres_recetas_pag = []
for i in links:
    url_bs4 = i
    pag_receta = requests.get(url_bs4)
    soup = BeautifulSoup(pag_receta.content, 'html.parser')
    # Guarda los tags <a> pertenecientes al titulo de cada receta por hoja
    informacion = soup.find_all('a', class_='titulo titulo--resultado')
    
    
    # Se obtiene el link guardado bajo el titulo de la receta
    for i in range(len(informacion)):
        receta = list(informacion)[i]
        link = receta.get("href")
        # Se aniade a la lista
        nombres_recetas_pag.append(link)

        
nombres_recetas_pag

['https://www.recetasgratis.net/receta-de-empanadas-de-salchicha-76505.html',
 'https://www.recetasgratis.net/receta-de-empanadas-de-berenjena-76504.html',
 'https://www.recetasgratis.net/receta-de-empanadas-de-bondiola-76499.html',
 'https://www.recetasgratis.net/articulo-tipos-de-tacos-76461.html',
 'https://www.recetasgratis.net/receta-de-ceviche-verde-76475.html',
 'https://www.recetasgratis.net/receta-de-empanada-calabresa-76474.html',
 'https://www.recetasgratis.net/receta-de-ceviche-de-queso-panela-76455.html',
 'https://www.recetasgratis.net/receta-de-ceviche-de-jicama-76454.html',
 'https://www.recetasgratis.net/receta-de-empanadas-fugazzeta-76467.html',
 'https://www.recetasgratis.net/receta-de-croquetas-sin-huevo-76457.html',
 'https://www.recetasgratis.net/receta-de-ceviche-acapulqueno-76453.html',
 'https://www.recetasgratis.net/receta-de-ceviche-tropical-76452.html',
 'https://www.recetasgratis.net/receta-de-pastel-de-tamales-76429.html',
 'https://www.recetasgratis.net/r

### Obtención de la info de la receta

(Esto tarda bastante ya que tiene que analizar +20000 recetas)

In [ ]:
recetas_n_comensales = []
recetas_duracion = []
recetas_dificultad = []
recetas_ingredientes = []
pasos_receta = []

# Recorre cada link de cada receta guardada en la lista
for j in nombres_recetas_pag:
    link_receta = j
    lista_nueva = []
    pag_receta_info = requests.get(link_receta)
    soup_receta_info = BeautifulSoup(pag_receta_info.content, 'html.parser')
    
    # Obtener los ingredientes de la receta
    
    # Algunas recetas resultan ser un top de recetas, por lo que si existe un indice se trata de un top, que en este caso
    # no se tomaran como recetas
    indice = soup_receta_info.find('div', class_="indice")
    if indice is None:
        
        # Obtener el numero de comensales
        n_comensales = soup_receta_info.find('span', {"class":"property comensales"})
        if n_comensales is None:
            n_comensales="Sin Informacion"
        else:
            recetas_n_comensales.append(n_comensales.get_text())
        
        # Obtener el tiempo de preparacion de la receta
        tiempo_duracion = soup_receta_info.find('span', {"class":"property duracion"})
        if tiempo_duracion is None:
            tiempo_duracion = "Sin Informacion"
        else:
            recetas_duracion.append(tiempo_duracion.get_text())
        
        # Obtener el nivel de dificultad de la receta
        dificultad_coccion = soup_receta_info.find('span', {"class":"property dificultad"})
        if dificultad_coccion is None:
            dificultad_coccion = "Sin Informacion"
        else:
            recetas_dificultad.append(dificultad_coccion.get_text())

        
        # Obtener la lista de ingredientes de la receta
        lista_ingredientes = soup_receta_info.find_all('li', class_="ingrediente")
        
        # De la lista de ingredientes obtener la etiqueta <label> que contiene el texto
        for i in range(len(lista_ingredientes)):
            lista_ingredient = list(lista_ingredientes)[i]
            nombres_ingredientes = lista_ingredient.find_all('label')
            
            # Se obtiene el texto crudo de todas las etiquetas <label>
            for j in range(len(nombres_ingredientes)):
                lista_nueva.append(nombres_ingredientes[j].get_text())

        # quitar los saltos de linea
        new_list = []
        for i in lista_nueva:
            new_list.append(i.strip())
        recetas_ingredientes.append(new_list)

        
        # Obtener los pasos de cada receta
        contenedor_pasos = soup_receta_info.find_all('div', class_="apartado")
        for i in range(len(contenedor_pasos)):
            pasos = contenedor_pasos[i].find_all('p')
            for j in pasos:
                pasos_receta.append(j.get_text())
    

In [88]:
pasos_receta

['El primer paso para esta receta de empanadas de salchicha consiste en mezclar la mayonesa con el ketchup y la salsa inglesa. Si lo prefieres, puedes usar directamente salsa golf, pero esta opción te permite hacerla a tu gusto y darle el toque distintivo de la salsa inglesa. También puedes reemplazarla por barbacoa, la cual combina perfectamente con las salchichas, al igual que la mostaza, o bien no utilizar ningún tipo de aderezo.',
 'Por otro lado, corta las salchichas al medio y luego en medias rueditas. Recomendamos elijas alguna ahumada, tipo alemana, en lugar de las clásicas para pancho, sobre todo si las preparas para adultos.',
 'Truco: al utilizarlas crudas deberás lavarte bien las manos luego de manipularlas y tener especial cuidado si estás preparando esta receta con niños.',
 'Mezcla las salchichas con la salsa golf',
 'Prepara también los discos de empanada, corta el queso mozzarella en cubos y ten a mano un cuenco con un poco de agua para ayudarte a cerrar las empanadas 

In [ ]:
####################################################################################

In [ ]:
# obtener los ingredientes

In [48]:
pag_receta_info = requests.get('https://www.recetasgratis.net/receta-de-arroz-caldoso-con-alcachofas-76016.html')
soup_receta_info = BeautifulSoup(pag_receta_info.content, 'html.parser')

In [52]:
lista_ingredientes = soup_receta_info.find_all('li', class_="ingrediente")

In [86]:
lista_nueva = []
for i in range(len(lista_ingredientes)):
    lista_ingredient = list(lista_ingredientes)[i]
    nombres_ingredientes = lista_ingredient.find_all('label')
    
    for j in range(len(nombres_ingredientes)):
        lista_nueva.append(nombres_ingredientes[j].get_text())

In [88]:
# quitar los newlines
new_list = []
for i in lista_nueva:
    new_list.append(i.strip())

In [89]:
new_list

['350 gramos de arroz bomba',
 '3 alcachofas',
 '1 cebolla',
 '1 trozo de puerro',
 '1 diente de ajo',
 '1½ litros de caldo de verduras',
 '3 cucharadas soperas de tomate triturado',
 '1 cucharada postre de azafrán',
 '1 chorro de aceite de oliva',
 '1 pizca de sal']

In [ ]:
# obtener los pasos

In [126]:
pasos_receta = []
contenedor_pasos = soup_receta_info.find_all('div', class_="apartado")
for i in range(len(contenedor_pasos)):
    pasos = contenedor_pasos[i].find_all('p')
    for j in pasos:
        pasos_receta.append(j.get_text())

In [132]:
pasos_receta

['Lave, seque y salpimente el pollo, por dentro y por fuera.',
 'Colóquelo en un refractario, agregue el caldo de pollo y póngalo en el horno precalentado a 200 grados C, durante 1 hora. 45 min., bañándolo frecuentemente con el caldo.',
 'A diez minutos de que este listo, cúbrale la pechuga con el queso.',
 'Mientras tanto, caliente la mantequilla, acitrone los echalotes picados y agregue los hongos cocidos por unos minutos, luego añada la mermelada disuelta con el liquido de la charola donde horneo el pollo y caliéntelo a punto de ebullición.',
 'Retire el pollo del horno, colóquelo en un platón de servicio y sírvalo bañado con la salsa de hongos.',
 'Si te ha gustado la receta de Pollo en salsa de hongos, te sugerimos que entres en nuestra categoría de Recetas de Pollo.',
 'Sube la foto de tu Receta de Pollo en salsa de hongos']

In [ ]:
#####################################################################################################

# Recetas online

### Obtención de las diferentes categorías

In [3]:
# Se realiza una peticion a la pagina web de recetas online
url_recetas_online = "https://www.recetasonline.net/recetas-de-cocina/"
pag_recetas_online = requests.get(url_recetas_online)
soup_recetas_online = BeautifulSoup(pag_recetas_online.content, 'html.parser')

In [4]:
# Se obtienen las diferentes categorias que se encuentran en el sub menu del navegador principal
sub_menu_categorias = soup_recetas_online.find('ul', class_="sub-menu")
# Se obtienen las etiquetas <a> de las diferentes categorias
link_sub_menu = sub_menu_categorias.find_all('a')

# Se obtienen los diferentes links de las categorias
links_categorias_recetas_online = []
for i in range(len(link_sub_menu)):
    link = link_sub_menu[i].get("href")
    links_categorias_recetas_online.append(link)
links_categorias_recetas_online    

['https://www.recetasonline.net/course/aperitivo-2/',
 'https://www.recetasonline.net/course/carne-2/',
 'https://www.recetasonline.net/course/elaboracion-2/',
 'https://www.recetasonline.net/course/entrante-2/',
 'https://www.recetasonline.net/course/pescado-2/',
 'https://www.recetasonline.net/course/postre-2/',
 'https://www.recetasonline.net/course/primero/',
 'https://www.recetasonline.net/course/segundo/',
 'https://www.recetasonline.net/course/unico/']

### Obtención de las diferentes hojas de cada categoría

In [5]:
paginas_totales_por_categoria = []
# Variable que comprueba si se ha llegado al final de las hojas para x categoria
final_recetas_online = False
contador = 0

# Recorre la lista de los links de las diferentes categorias, para obtener el numero total de paginas pertenecientes a esa
# categoria
for categoria in links_categorias_recetas_online:
    url_categoria_recetas_online = categoria
    
    # Comprueba que se ha llegado al final de las categorias existentes
    if contador == len(links_categorias_recetas_online)-1:
        final_recetas_online = True
    else:
        final_recetas_online = False
        
    # Por cada categoria revisa si existe una siguiente hoja
    paginas_totales_por_categoria.append(url_categoria_recetas_online)
    while (final_recetas_online==False):
        pag_categoria_recetas_online = requests.get(url_categoria_recetas_online)
        soup_pag_categoria_recetas_online = BeautifulSoup(pag_categoria_recetas_online.content, 'html.parser')
    
        # comprobar si exiten mas hojas para la x categoria
        mark_next_page = soup_pag_categoria_recetas_online.find('a', class_="next page-numbers")
    
        num_paginas_categoria = 1
        # Si no existe significa que se ha llegado al final de las hojas de x categoria
        if mark_next_page is None:
            final_recetas_online = True
        else:
            # Si existe se obtiene el link de la siguienta hoja de recetas para x categoria
            mark_next_page = soup_pag_categoria_recetas_online.find('a', class_="next page-numbers").get('href')
            url_categoria_recetas_online = mark_next_page
            paginas_totales_por_categoria.append(url_categoria_recetas_online)
        
    contador+=1

In [17]:
paginas_totales_por_categoria

['https://www.recetasonline.net/course/aperitivo-2/',
 'https://www.recetasonline.net/course/aperitivo-2/page/2/',
 'https://www.recetasonline.net/course/carne-2/',
 'https://www.recetasonline.net/course/carne-2/page/2/',
 'https://www.recetasonline.net/course/carne-2/page/3/',
 'https://www.recetasonline.net/course/elaboracion-2/',
 'https://www.recetasonline.net/course/elaboracion-2/page/2/',
 'https://www.recetasonline.net/course/elaboracion-2/page/3/',
 'https://www.recetasonline.net/course/entrante-2/',
 'https://www.recetasonline.net/course/entrante-2/page/2/',
 'https://www.recetasonline.net/course/entrante-2/page/3/',
 'https://www.recetasonline.net/course/pescado-2/',
 'https://www.recetasonline.net/course/pescado-2/page/2/',
 'https://www.recetasonline.net/course/postre-2/',
 'https://www.recetasonline.net/course/postre-2/page/2/',
 'https://www.recetasonline.net/course/postre-2/page/3/',
 'https://www.recetasonline.net/course/primero/',
 'https://www.recetasonline.net/course

In [23]:
# Lista que guardara las recetas que se encuentran bajo una categoria
recetas_x_categoria = []

# Se recorre cada pagina guardada en la lista de "paginas_totales_por_categoria"
for pagina in paginas_totales_por_categoria:
    url_link_categoria_pagina = pagina
    pag_link_categoria_ = requests.get(url_link_categoria_pagina, headers=header)
    soup_pag_link_categoria_ = BeautifulSoup(pag_link_categoria_.content, 'html.parser')
    # Se obtienen los titulos de las recetas
    titulo_receta_recetas_online = soup_pag_link_categoria_.find_all('h2', class_="entry-title")
    
    # Se obtienen los links que se encuentran en los titulos de cada receta
    for i in range(len(titulo_receta_recetas_online)):
        link_titulo_receta_recetas_online = list(titulo_receta_recetas_online)[i].find('a').get('href')
        recetas_x_categoria.append(link_titulo_receta_recetas_online)

### Obtención de la información de cada receta

In [ ]:
comensales_porReceta_recetas_online = []
tiempo_porReceta_recetas_online = []
ingredientes_porReceta_recetas_online = []
pasos_porReceta_recetas_online = []

# Se recorre la lista de recetas obtenida anteriormente
for link_receta_ in recetas_x_categoria:
    pag_receta_informacion_recetas_online = requests.get(link_receta_)
    soup_pag_receta_info_recetas_online = BeautifulSoup(pag_receta_informacion_recetas_online.content, 'html.parser')
    
    # Numero de comensales
    list_n_comensales_recetas_online = soup_pag_receta_info_recetas_online.find('li', class_="servings")
    if list_n_comensales_recetas_online is None:
        n_comensales_recetas_online = "Sin Informacion"
    else:
        n_comensales_recetas_online = list_n_comensales_recetas_online.find('span').get_text()
    comensales_porReceta_recetas_online.append(n_comensales_recetas_online)
    
    # Tiempo de preparacion
    list_tiempo_receta_recetas_online = soup_pag_receta_info_recetas_online.find('li', class_="ready-in")
    if list_tiempo_receta_recetas_online is None:
        tiempo_receta_recetas_online = "Sin Informacion"
    else:
        tiempo_receta_recetas_online = list_tiempo_receta_recetas_online.find('span').get_text()
    tiempo_porReceta_recetas_online.append(tiempo_receta_recetas_online)
    
    # Lista de ingredientes
    list_ingredientes_receta_recetas_online = soup_pag_receta_info_recetas_online.find_all('li', class_="ingredient")
    
    lista_temp = []
    for ingredient in range(len(list_ingredientes_receta_recetas_online)):
        lista_temp.append(list_ingredientes_receta_recetas_online[ingredient].get_text())
    ingredientes_porReceta_recetas_online.append(lista_temp)
    
    
    # Lista de pasos a seguir
    list_pasos_recetas_online = soup_pag_receta_info_recetas_online.find_all('p', class_="instructions")
    lista_temp_pasos = []
    for paso in range(len(list_pasos_recetas_online)):
        lista_temp_pasos.append(list_pasos_recetas_online[paso].get_text())
    pasos_porReceta_recetas_online.append(lista_temp_pasos)

# Hogarmania

### Obtención de los links de las diferentes categorías

In [6]:
# Link original de la pagina web, ya que los links de las recetas no se guardan la ruta completa
url_hogarmania_principal = "https://www.hogarmania.com"
# Link de la pagina web para su categoria de recetas
url_hogarmania = "https://www.hogarmania.com/cocina/recetas/"
pagina_hogarmania = requests.get(url_hogarmania, headers=header)
soup_hogarmania = BeautifulSoup(pagina_hogarmania.content, 'html.parser')

lista_links_titulos_categoria_hogarmania = []

# Obtener los diferentes links pertenecientes a cada categoria
# Se han quitado los 2 titulos utlimos porque son consejos
titulos_categorias_hogarmania = soup_hogarmania.find_all('h2', class_="m-titulo")
for titulo in range(len(titulos_categorias_hogarmania[:-3])):
    lista_titulos_hogarmania = list(titulos_categorias_hogarmania)[titulo]
    link_titulo_categoria_hogarmania = lista_titulos_hogarmania.find('a').get('href')
    # Se une el link horiginal junto al link obtenido de cada categoria, para que a la hora de navegar se tenga la
    # url completa
    link_titulo_final = url_hogarmania_principal+link_titulo_categoria_hogarmania
    lista_links_titulos_categoria_hogarmania.append(link_titulo_final)

In [7]:
lista_links_titulos_categoria_hogarmania

['https://www.hogarmania.com/cocina/recetas/postres/',
 'https://www.hogarmania.com/cocina/recetas/carnes/',
 'https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/',
 'https://www.hogarmania.com/cocina/recetas/pescados-mariscos/',
 'https://www.hogarmania.com/cocina/recetas/pastas-pizzas/',
 'https://www.hogarmania.com/cocina/recetas/legumbres/',
 'https://www.hogarmania.com/cocina/recetas/sopas-cremas/',
 'https://www.hogarmania.com/cocina/recetas/arroces-cereales/',
 'https://www.hogarmania.com/cocina/recetas/aperitivos/',
 'https://www.hogarmania.com/cocina/recetas/salsas/',
 'https://www.hogarmania.com/cocina/recetas/huevos/',
 'https://www.hogarmania.com/cocina/recetas/setas-hongos/',
 'https://www.hogarmania.com/cocina/recetas/cocteles/']

### Obtención de las diferentes hojas de cada categoría

In [8]:
paginas_totales_todasCategorias_hogarmania = []
# Variable que comprueba que el paginador sea un numero o un signo, ya que en este sitio el boton para pasar de hoja
# no tiene un identificador por lo que solo tienen un icono "»", el cual es el que va comparando con otros
# para saber si existe una pagina siguiente para una cierta categoria
paginador_numero = False
contador_hogarmania = 0

# Se recorren las diferentes categorias
for categoria in lista_links_titulos_categoria_hogarmania:
    url_prueba = categoria
    
    # Se comprueba si se ha acabado de recorrer todas las categorias
    if contador_hogarmania == (len(lista_links_titulos_categoria_hogarmania)-1):
        paginador_numero = True
    else:
        paginador_numero = False
    
    paginas_totales_todasCategorias_hogarmania.append(url_prueba)
    # Bucle que comprueba si existen mas paginas para cierta categoria, obteniendo su respectivo link
    while (paginador_numero==False):

        pagina_categoria_hogarmania = requests.get(url_prueba, headers=header)
        soup_categoria_hogarmania = BeautifulSoup(pagina_categoria_hogarmania.content, 'html.parser')

        # Se obtienen el contenedor que guarda la paginacion para la categoria deseada
        paginador_siguiente_hogarmania = soup_categoria_hogarmania.find('ul', class_="pagination")
        link_paginador_hogarmania = paginador_siguiente_hogarmania.find_all('a')

        for i in range(len(link_paginador_hogarmania)):
            # Se comprueba que el texto que aperece en el paginador sea igual al signo que representa la siguiente pagina
            if (link_paginador_hogarmania[i].get_text()) == "»":
                # Si existe se obtiene el link
                link_semicompleto = link_paginador_hogarmania[i].get('href')
                # Se une el link original y el nuevo link obtenido
                link_completo = url_hogarmania_principal+link_semicompleto
                url_prueba = link_completo
                paginas_totales_todasCategorias_hogarmania.append(url_prueba)
                paginador_numero = False

            else:
                # Si el paginador no es el signo de "siguiente"
                paginador_numero = True
        if paginador_numero == True:
            print("")
        contador_hogarmania+=1


https://www.hogarmania.com/cocina/recetas/postres/pagina/2
https://www.hogarmania.com/cocina/recetas/postres/pagina/3
https://www.hogarmania.com/cocina/recetas/postres/pagina/4
https://www.hogarmania.com/cocina/recetas/postres/pagina/5
https://www.hogarmania.com/cocina/recetas/postres/pagina/6
https://www.hogarmania.com/cocina/recetas/postres/pagina/7
https://www.hogarmania.com/cocina/recetas/postres/pagina/8
https://www.hogarmania.com/cocina/recetas/postres/pagina/9
https://www.hogarmania.com/cocina/recetas/postres/pagina/10
https://www.hogarmania.com/cocina/recetas/postres/pagina/11
https://www.hogarmania.com/cocina/recetas/postres/pagina/12
https://www.hogarmania.com/cocina/recetas/postres/pagina/13
https://www.hogarmania.com/cocina/recetas/postres/pagina/14
https://www.hogarmania.com/cocina/recetas/postres/pagina/15
https://www.hogarmania.com/cocina/recetas/postres/pagina/16
https://www.hogarmania.com/cocina/recetas/postres/pagina/17
https://www.hogarmania.com/cocina/recetas/postre

https://www.hogarmania.com/cocina/recetas/postres/pagina/139
https://www.hogarmania.com/cocina/recetas/postres/pagina/140
La ultima hoja es un numero
https://www.hogarmania.com/cocina/recetas/carnes/pagina/2
https://www.hogarmania.com/cocina/recetas/carnes/pagina/3
https://www.hogarmania.com/cocina/recetas/carnes/pagina/4
https://www.hogarmania.com/cocina/recetas/carnes/pagina/5
https://www.hogarmania.com/cocina/recetas/carnes/pagina/6
https://www.hogarmania.com/cocina/recetas/carnes/pagina/7
https://www.hogarmania.com/cocina/recetas/carnes/pagina/8
https://www.hogarmania.com/cocina/recetas/carnes/pagina/9
https://www.hogarmania.com/cocina/recetas/carnes/pagina/10
https://www.hogarmania.com/cocina/recetas/carnes/pagina/11
https://www.hogarmania.com/cocina/recetas/carnes/pagina/12
https://www.hogarmania.com/cocina/recetas/carnes/pagina/13
https://www.hogarmania.com/cocina/recetas/carnes/pagina/14
https://www.hogarmania.com/cocina/recetas/carnes/pagina/15
https://www.hogarmania.com/cocin

https://www.hogarmania.com/cocina/recetas/carnes/pagina/138
https://www.hogarmania.com/cocina/recetas/carnes/pagina/139
https://www.hogarmania.com/cocina/recetas/carnes/pagina/140
https://www.hogarmania.com/cocina/recetas/carnes/pagina/141
https://www.hogarmania.com/cocina/recetas/carnes/pagina/142
https://www.hogarmania.com/cocina/recetas/carnes/pagina/143
https://www.hogarmania.com/cocina/recetas/carnes/pagina/144
https://www.hogarmania.com/cocina/recetas/carnes/pagina/145
https://www.hogarmania.com/cocina/recetas/carnes/pagina/146
La ultima hoja es un numero
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/2
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/3
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/4
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/5
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/6
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/7
https://www.

https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/110
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/111
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/112
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/113
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/114
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/115
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/116
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/117
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/118
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/119
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/120
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/121
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pagina/122
https://www.hogarmania.com/cocina/recetas/ensaladas-verduras/pag

https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/37
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/38
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/39
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/40
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/41
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/42
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/43
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/44
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/45
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/46
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/47
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/48
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/49
https://www.hogarmania.com/cocina/recetas/pescados-mariscos/pagina/50
https://www.hogarman

https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/12
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/13
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/14
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/15
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/16
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/17
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/18
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/19
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/20
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/21
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/22
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/23
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/24
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/25
https://www.hogarmania.com/cocina/recetas/pastas-pizzas/pagina/26
https://ww

https://www.hogarmania.com/cocina/recetas/arroces-cereales/pagina/26
https://www.hogarmania.com/cocina/recetas/arroces-cereales/pagina/27
https://www.hogarmania.com/cocina/recetas/arroces-cereales/pagina/28
https://www.hogarmania.com/cocina/recetas/arroces-cereales/pagina/29
https://www.hogarmania.com/cocina/recetas/arroces-cereales/pagina/30
https://www.hogarmania.com/cocina/recetas/arroces-cereales/pagina/31
https://www.hogarmania.com/cocina/recetas/arroces-cereales/pagina/32
La ultima hoja es un numero
https://www.hogarmania.com/cocina/recetas/aperitivos/pagina/2
https://www.hogarmania.com/cocina/recetas/aperitivos/pagina/3
https://www.hogarmania.com/cocina/recetas/aperitivos/pagina/4
https://www.hogarmania.com/cocina/recetas/aperitivos/pagina/5
https://www.hogarmania.com/cocina/recetas/aperitivos/pagina/6
https://www.hogarmania.com/cocina/recetas/aperitivos/pagina/7
https://www.hogarmania.com/cocina/recetas/aperitivos/pagina/8
https://www.hogarmania.com/cocina/recetas/aperitivos/pa

In [9]:
paginas_totales_todasCategorias_hogarmania

['https://www.hogarmania.com/cocina/recetas/postres/',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/2',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/3',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/4',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/5',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/6',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/7',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/8',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/9',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/10',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/11',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/12',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/13',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/14',
 'https://www.hogarmania.com/cocina/recetas/postres/pagina/15',
 'https://www.hogarmania.com/cocina/recetas/postres/pagin

### Obtención de las diferentes recetas de todas las hojas de x categoría

In [11]:
# Lista que guarda todas las recetas de cada categoria
link_todasRecetasPorCategoria = []

# Recorre todas las hojas de una categoria para obtener todas las recetas bajo esa categoria
for hoja in paginas_totales_todasCategorias_hogarmania:
    link_hojaDeRecetas_categoria_hogarmania = hoja
    pag_linkHojaRecetas_categoria_ = requests.get(link_hojaDeRecetas_categoria_hogarmania, headers=header)
    soup_pagHojaRecetas_categoria = BeautifulSoup(pag_linkHojaRecetas_categoria_.content, 'html.parser')
    # Se obtiene el contenedor que guarda el link de la receta
    contenedor_titulos_recetas_hogarmania = soup_pagHojaRecetas_categoria.find('div', class_="especial listado")
    articulo_recetas = contenedor_titulos_recetas_hogarmania.find_all('article', class_="modulo")
    
    # Se recorre el contenedor que guarda la informacion de la receta
    for articulo in range(len(articulo_recetas)):
        lista_articulo_recetas = list(articulo_recetas)[articulo]
        link_receta = lista_articulo_recetas.find_all('h2', class_="m-titulo")
        
        # Se obtiene el link de la recetas
        for contenido in range(len(link_receta)):
            lista_contenido = list(link_receta)[contenido]
            links_titulo_receta_hogarmaniaCategoria = lista_contenido.find_all('a')
            urls_titulos_receta_hogarmaniaCategoria = list(links_titulo_receta_hogarmaniaCategoria)[contenido]
            url_todasRecetasPorCategoria = url_hogarmania_principal+urls_titulos_receta_hogarmaniaCategoria.get('href')
            
            link_todasRecetasPorCategoria.append(url_todasRecetasPorCategoria)

In [12]:
link_todasRecetasPorCategoria

['https://www.hogarmania.com/cocina/recetas/postres/orejas-carnaval-sin-gluten.html',
 'https://www.hogarmania.com/cocina/recetas/postres/tarta-de-naranja-y-queso.html',
 'https://www.hogarmania.com/cocina/recetas/postres/tarta-banoffee.html',
 'https://www.hogarmania.com/cocina/recetas/postres/chocotarta-argentina.html',
 'https://www.hogarmania.com/cocina/recetas/postres/espiga-de-hojaldre-y-crema-de-chocolate.html',
 'https://www.hogarmania.com/cocina/recetas/postres/brownie-chocolate-cafe.html',
 'https://www.hogarmania.com/cocina/recetas/postres/baklava-de-pistachos-y-crema-de-queso.html',
 'https://www.hogarmania.com/cocina/recetas/postres/leche-frita-vegana.html',
 'https://www.hogarmania.com/cocina/recetas/postres/natillas-de-chocolate.html',
 'https://www.hogarmania.com/cocina/recetas/postres/batido-manzana-kiwi-platano.html',
 'https://www.hogarmania.com/cocina/recetas/postres/gelatina-naranja.html',
 'https://www.hogarmania.com/cocina/recetas/postres/galette-rois-pastel-reye

### Obtención de la información de cada receta

In [75]:
tiempo_total_RecetasHogarmania = []
ingredientes_total_RecetasHogarmania = []
pasos_RecetaHogarmania = []
# Lista de prueba para no tener que ejecutar todo
#lin_ = []
#lin_.append('https://www.hogarmania.com/cocina/recetas/postres/orejas-carnaval-sin-gluten.html')
#lin_.append('https://www.hogarmania.com/cocina/recetas/postres/tarta-de-naranja-y-queso.html')

# Se recorre todas las recetas de todas las categorias
for receta in link_todasRecetasPorCategoria:
    url_receta = receta
    pagina_receta_hogarmania = requests.get(url_receta, headers=header)
    soup_receta_hogarmania = BeautifulSoup(pagina_receta_hogarmania.content, 'html.parser')
    
    # Obtener el tiempo de preparacion
    contenedor_tiempo_total_receta_hogarmania = soup_receta_hogarmania.find_all('b')
    if contenedor_tiempo_total_receta_hogarmania is None:
        tiempo_total_RecetasHogarmania.append('Sin Informacion')
    else:
        # El tiempo se guarda junto a otro tiempo, por lo tanto se deben separar para obtener el tiempo final de preparacion
        for tiempos in range(len(contenedor_tiempo_total_receta_hogarmania)):
            tiempo_totalPreparacion_recetaHogarmania = contenedor_tiempo_total_receta_hogarmania[tiempos].get_text()

        tiempo_total_RecetasHogarmania.append(tiempo_totalPreparacion_recetaHogarmania)
    
    
    # Obtener los diferentes ingredientes
    contenedor_ingredientes_recetaHogarmania = soup_receta_hogarmania.find('ul', class_="ingredientes")
    lista_ingredientes_tmp = []
    # Se recorre la lista de los ingredientes para obtener el texto
    for ingrediente_hogarmania in contenedor_ingredientes_recetaHogarmania:
        ingrediente_deLaLista = ingrediente_hogarmania.get_text()
        lista_ingredientes_tmp.append(ingrediente_deLaLista)
    ingredientes_total_RecetasHogarmania.append(lista_ingredientes_tmp)
    
    
    # Obtener los pasos a seguir
    lista_identificadores_h2 = []
    # Obtener el id del titulo, ya que la parte de elaboracion no tiene identificador
    for titulos_h2_hogarmania in soup_receta_hogarmania.find_all('h2'):
        lista_identificadores_h2.append(titulos_h2_hogarmania.get('id'))
        
    # Como el contenido de los pasos no tiene un identificador, para obtener tal informacion, se obtiene el identificador
    # del titulo que da comienzo a la elaboracion del plato y el siguiente titulo que da comienza a otra fase
    
    # Titulo de comienzo de la elaboracion
    start_h2_recetaElaboracion_hogarmania = soup_receta_hogarmania.find('h2', {'id': lista_identificadores_h2[1]})
    # Titulo del siguiente parrafo de contenido
    end_h2_recetaElaboracion_hogarmania = soup_receta_hogarmania.find('h2', {'id': lista_identificadores_h2[2]})
    content = ""
    # Se obtiene las siguientes partes de la web que siguen tras el titulo de comienzo de la elaboracion
    item = start_h2_recetaElaboracion_hogarmania.nextSibling
    
    # Se une el contenido y se aniade a una lista
    lista_contenido_tmp = list()
    # Se compara que las siguientes partes del titulo de comienzo no sean igual al titulo del final
    while item != end_h2_recetaElaboracion_hogarmania:
        content = item
        item = item.nextSibling
        lista_contenido_tmp.append(content)
    
    
    # Se van a eliminar todas aquellas etiquetas que no sean de texto
    lista_pasos_tmp = []
    for tag in range(len(lista_contenido_tmp)):
        
        tags = list(lista_contenido_tmp)[tag]

        # Se comprueba que el tipo sea un bs4 tag element
        if (type(tags))==(type(start_h2_recetaElaboracion_hogarmania)):
            
            # Se obtiene el texto, se reliza un try ya que la web tiene elementos que no son del tipo tag, y puede dar error
            # a la hora de intentar rescatar el texto que se encuentra en el
            try:
                contenido_pasos = tags.get_text()
                lista_pasos_tmp.append(contenido_pasos)
                
            except Exception as e:
                print("no se puede obtener el texto")
            
        else:
            print("")
    pasos_RecetaHogarmania.append(lista_pasos_tmp)
    
pasos_RecetaHogarmania

[['Bate los huevos en un bol y añádele azúcar, mantequilla y sal. Disuelve la levadura en un vaso de agua templada e incorpórala también. Mezcla todo bien. ',
  'Añade el vaso de anís, la ralladura de limón y las gotas de esencia de anís. Remueve para que se integren bien todos los ingredientes. ',
  'Coloca la harina sobre la mesa y hazle un hueco por la mitad como si fuera un volcán. Añade la mezcla preparada previamente y empieza a amasarlo poco a poco con las manos hasta conseguir una masa elástica. ',
  'Haz una bola y colócala en un bol tapada con film. Deja que repose en la nevera durante 2 horas aproximadamente. Una vez reposada, corta en trozos pequeños y estíralos con ayuda de un rodillo hasta que queden finitos. Dales la forma de triángulo e introdúcelas de nuevo en la nevera media hora aproximadamente. ',
  'Calienta abundante aceite en una sartén y cuando coja calor, añade las orejas de carnaval sin gluten. Cocínalas por ambos lados hasta que queden bien doraditas. ',
  'S